In [ ]:
from timeseries_fastai.imports import *
from timeseries_fastai.data import *
from timeseries_fastai.core import *
from timeseries_fastai.models import *
from timeseries_fastai.tabular import *

from ucr import *

In [ ]:
PATH = get_ucr()

In [ ]:
PATH.ls()

(#135) [Path('/home/tc256760/.fastai/data/Univariate2018_arff/PowerCons'),Path('/home/tc256760/.fastai/data/Univariate2018_arff/PhalangesOutlinesCorrect'),Path('/home/tc256760/.fastai/data/Univariate2018_arff/Herring'),Path('/home/tc256760/.fastai/data/Univariate2018_arff/ElectricDevices'),Path('/home/tc256760/.fastai/data/Univariate2018_arff/CBF'),Path('/home/tc256760/.fastai/data/Univariate2018_arff/Plane'),Path('/home/tc256760/.fastai/data/Univariate2018_arff/DataSummaryExpanded_v03.xlsx'),Path('/home/tc256760/.fastai/data/Univariate2018_arff/MiddlePhalanxTW'),Path('/home/tc256760/.fastai/data/Univariate2018_arff/ECG5000'),Path('/home/tc256760/.fastai/data/Univariate2018_arff/CricketZ')...]

In [ ]:
NON_NAN_TASKS[0:5]

['ACSF1', 'Adiac', 'ArrowHead', 'BME', 'Beef']

In [ ]:
def max_bs(N):
    N = N//6
    k=1
    while (N//2**k)>1: k+=1
    return min(2**k, 32)

In [ ]:
def get_dls(path, task, bs=None, workers=None):
    df_train, df_test = load_df_ucr(path, task)
    bs = ifnone(bs, max_bs(len(df_train)))
    x_cols = df_train.columns[0:-1].to_list()
    return TSDataLoaders.from_dfs(df_train, df_test, x_cols=x_cols, label_col='target', 
                                  bs=bs, num_workers=workers)

In [ ]:
dls = get_dls(PATH, 'Adiac')

Loading files from: /home/tc256760/.fastai/data/Univariate2018_arff/Adiac


In [ ]:
def get_dls2(path, task, bs=None, workers=None):
    df_train, df_test = load_df_ucr(path, task)
    bs = ifnone(bs, max_bs(len(df_train)))
    x_cols = df_train.columns[0:-1].to_list()
    
    df_main = stack_train_valid(df_train, df_test)
    splits=[range_of(df_train), list(range(len(df_train), len(df_main)))]
    to = TSPandas(df_main, [Normalize], x_names=x_cols, y_names='target', splits=splits)
    
    return to.dataloaders(bs, 2*bs)

In [ ]:
dls = get_dls2(PATH, 'Adiac')

Loading files from: /home/tc256760/.fastai/data/Univariate2018_arff/Adiac


In [ ]:
dls.c

37

In [ ]:
x,y = dls.train.one_batch()

In [ ]:
x.shape

torch.Size([32, 1, 176])

In [ ]:
def get_model(dls, arch):
    num_classes = dls.c
    arch = arch.lower()
    if arch=='resnet':     model = create_resnet(1, num_classes, conv_sizes=[64, 128, 256])
    elif arch=='fcn':      model = create_fcn(1, num_classes, ks=9, conv_sizes=[128, 256, 128])
    elif arch=='mlp':      model = create_mlp(dls.train.one_batch()[0].shape[-1], num_classes)
    elif arch=='inception':model = create_inception(1, num_classes)
    else: 
        print('Please chosse a model in [resnet, FCN, MLP, inception]')
        return None
    return model

In [ ]:
LabelSmoothingCrossEntropy??

In [ ]:
def train_task(path, task='Adiac', arch='resnet', epochs=40, lr=5e-4):
    "trains arch over task with params"
    dls = get_dls2(path, task)
    model = get_model(dls, arch)
    cbs =  MixUp(0.2)
    learn = Learner(dls, model=get_model(dls, arch), opt_func=Adam, \
            metrics=[accuracy])
    learn.fit_one_cycle(epochs, lr, cbs=cbs)                          
    return learn.to_fp16()

In [ ]:
learn = train_task(PATH, 'Adiac', arch='mlp', epochs=4)

Loading files from: /home/tc256760/.fastai/data/Univariate2018_arff/Adiac


epoch,train_loss,valid_loss,accuracy,time
0,3.678888,3.401350,0.173913,00:00
1,3.299784,2.781090,0.368286,00:00
2,2.972312,2.366914,0.416880,00:00
3,2.798428,2.240401,0.421995,00:00


In [ ]:
def compute_metrics(learn):
    "compute oguiza Metrics on UCR"
    results = np.array(learn.recorder.values)
    acc_ = results[-1,-1]
    accmax_ = results[:, -1].max()
    loss_ = results[-1,0]
    val_loss_ = results[-1,1]
    return acc_, accmax_, loss_, val_loss_ 

In [ ]:
learn = train_task(PATH, NON_NAN_TASKS[0], epochs=4)

Loading files from: /home/tc256760/.fastai/data/Univariate2018_arff/ACSF1


epoch,train_loss,valid_loss,accuracy,time
0,2.388999,2.242882,0.170000,00:00
1,1.849001,1.666387,0.510000,00:00
2,1.500950,1.252819,0.660000,00:00
3,1.297396,1.008192,0.710000,00:00


In [ ]:
def run_tasks(tasks, arch='resnet', lr=1e-3, epochs=1):
    results = [compute_metrics(train_task(PATH, task, arch, epochs, lr)) for task in tasks]
    return pd.DataFrame(data=results, columns=['acc', 'acc_max', 'train_loss', 'val_loss'], index=tasks)

In [ ]:
run_tasks(NON_NAN_TASKS)

Loading files from: /home/tc256760/.fastai/data/Univariate_arff/Adiac


epoch,train_loss,valid_loss,accuracy,time
0,2.839430,3.992089,0.025575,00:01


Loading files from: /home/tc256760/.fastai/data/Univariate_arff/ArrowHead


epoch,train_loss,valid_loss,accuracy,time
0,1.114421,1.626481,0.394286,00:01


Loading files from: /home/tc256760/.fastai/data/Univariate_arff/Beef


...

epoch,train_loss,valid_loss,accuracy,time
0,0.548429,0.660219,0.485834,00:03


Loading files from: /home/tc256760/.fastai/data/Univariate_arff/StarLightCurves


epoch,train_loss,valid_loss,accuracy,time
0,0.282737,1.930876,0.418650,00:13


Loading files from: /home/tc256760/.fastai/data/Univariate_arff/Strawberry


RuntimeError: CUDA out of memory. Tried to allocate 5.04 GiB (GPU 0; 10.76 GiB total capacity; 262.99 MiB already allocated; 5.06 GiB free; 464.00 MiB reserved in total by PyTorch)